In [ ]:
import numpy as np
import pandas as pd
from ast import literal_eval
import itertools



# Distance calc

In [ ]:


# Calculate Picker Route Distance between two locations
def distance_picking(Loc1, Loc2, y_low, y_high):

    # Start Point
    x1, y1 = Loc1[0], Loc1[1]
    # End Point
    x2, y2 = Loc2[0], Loc2[1]
    # Distance x-axis
    distance_x = abs(x2 - x1)
    # Distance y-axis
    if x1 == x2:
        distance_y1 = abs(y2 - y1)
        distance_y2 = distance_y1
    else:
        distance_y1 = (y_high - y1) + (y_high - y2)
        distance_y2 = (y1 - y_low) + (y2 - y_low)
    # Minimum distance on y-axis 
    distance_y = min(distance_y1, distance_y2)    
    # Total distance
    distance = distance_x + distance_y

    return distance

## find next closest locations

In [ ]:
# Find closest next location 
def next_location(start_loc, list_locs, y_low, y_high):

    # Distance to every next points candidate
    list_dist = [distance_picking(start_loc, i, y_low, y_high) for i in list_locs]
    # Minimum Distance 
    distance_next = min(list_dist)
    # Location of minimum distance
    index_min = list_dist.index(min(list_dist))
    next_loc = list_locs[index_min] # Next location is the first location with distance = min (**)
    list_locs.remove(next_loc)      # Next location is removed from the list of candidates
    
    return list_locs, start_loc, next_loc, distance_next

## Create your picking route and calculate the total walking distance

In [1]:
# Calculate total distance to cover for a list of locations
def create_picking_route(origin_loc, list_locs, y_low, y_high):

    # Total distance variable
    wave_distance = 0
    # Current location variable 
    start_loc = origin_loc
    # Store routes
    list_chemin = []
    list_chemin.append(start_loc)
    
    while len(list_locs) > 0: # Looping until all locations are picked
        # Going to next location
        list_locs, start_loc, next_loc, distance_next = next_location(start_loc, list_locs, y_low, y_high)
        # Update start_loc 
        start_loc = next_loc
        list_chemin.append(start_loc)
        # Update distance
        wave_distance = wave_distance + distance_next 

    # Final distance from last storage location to origin
    wave_distance = wave_distance + distance_picking(start_loc, origin_loc, y_low, y_high)
    list_chemin.append(origin_loc)
    
    return wave_distance, list_chemin

# Mapping orders by wave number 

In [ ]:
# Mapping orders by wave number 
def orderlines_mapping(df_orderlines, orders_number):

	# Order dataframe by timeframe
	df_orderlines = df_orderlines.sort_values(by='TimeStamp', ascending = True)
	# Unique order numbers list
	list_orders = df_orderlines.OrderNumber.unique()
	# Dictionnary for mapping
	dict_map = dict(zip(list_orders, [i for i in range(1, len(list_orders))]))
	# Order ID mapping
	df_orderlines['OrderID'] = df_orderlines['OrderNumber'].map(dict_map)
	# Grouping Orders by Wave of orders_number 
	df_orderlines['WaveID'] = (df_orderlines.OrderID%orders_number == 0).shift(1).fillna(0).cumsum()
	# Counting number of Waves
	waves_number = df_orderlines.WaveID.max() + 1

	return df_orderlines, waves_number

In [ ]:

# Getting storage locations to cover for a wave of orders
def locations_listing(df_orderlines, wave_id):

	# Filter by wave_id
	df = df_orderlines[df_orderlines.WaveID == wave_id]
	# Create coordinates listing
	list_locs = list(df['Coord'].apply(lambda t: literal_eval(t)).values)
	list_locs.sort()
	# Get unique Unique coordinates
	list_locs = list(k for k,_ in itertools.groupby(list_locs))
	n_locs = len(list_locs)

	return list_locs, n_locs

# Main

In [ ]:

import numpy as np
import pandas as pd
from calculate_distance import distance_picking, next_location, create_picking_route
from wave_creation import orderlines_mapping, locations_listing
from pre_processing import import_dataset

# Import dataframe with orderlines
df_orderlines = import_dataset('df_mois12-2018-P.xlsx')

# Parameters
y_low, y_high = 5.5, 50 		# Alley Coordinates on y-axis
Loc_orn = [0, y_low] 			# Origin Location
orders_number = 3 			# Number of orders per wave

# Create lists to store results
list_wid, list_dst, list_route, list_ord = [], [], [], []

# Function 
def simulation_wave(y_low, y_high, orders_number, df_orderlines, list_wid, list_dst, list_route, list_ord):

	# Create variable to store total distance
	distance_route = 0 
	# Create waves
	df_orderlines, waves_number = orderlines_mapping(df_orderlines, orders_number)
	# Loop all waves
	for wave_id in range(waves_number):
		# Listing of all locations for this wave 
		list_locs, n_locs = locations_listing(df_orderlines, wave_id)
		# Results
		wave_distance, list_chemin = create_picking_route(Loc_orn, list_locs, y_low, y_high)
		distance_route = distance_route + wave_distance
		# Append lists of results 
		list_wid.append(wave_id)
		list_dst.append(wave_distance)
		list_route.append(list_chemin)
		list_ord.append(orders_number)

	return list_wid, list_dst, list_route, list_ord, distance_route

# Test several values of orders per wave
for orders_number in range(1, 10):
	list_wid, list_dst, list_route, list_ord, distance_route = simulation_wave(y_low, y_high, orders_number, df_orderlines, list_wid, list_dst, list_route, list_ord)
	print("Total distance covered for {} orders/wave: {:,} m".format(orders_number, distance_route))

# Create df for results
df_results = pd.DataFrame({'Wave_Number': list_wid,
			   'Distance_Route': list_dst,
			   'Chemins': list_route,
			   'OrderPerWave': list_ord})

print(df_results.head())